In [0]:
create or refresh streaming table bronze_addresses
as
select *,_metadata.file_path as input_file_name,current_timestamp() as ingestion_time 
from cloud_files('/Volumes/circuitbox/landing/operationaldata/addresses/','csv',map('cloudFiles.inferColumnTypes' , 'true'))

In [0]:
create or refresh streaming table silver_addresses_clean(
  constraint validate_customer_id expect(customer_id is not null) on violation fail update,
  constraint validate_addresses_id expect(address_line_1 is not null) on violation drop row,
  constraint validate_postcode expect(length(postcode) >= 5)
)
tblproperties ("quality" = "true")
comment "this is sliver clean table"
as
select 
customer_id,
address_line_1,
city,
state,
postcode,
cast(created_date as date) as created_date
from stream(live.bronze_addresses)

In [0]:
create or refresh streaming table silver_addresses
comment "this is silver table"
tblproperties ("quality" = "silver")

In [0]:
CREATE FLOW address_flow_Sql AS AUTO CDC INTO silver_addresses
FROM STREAM(live.silver_addresses_clean)
KEYS (customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 2